# Exploring the Data

- [Imports](#imports)   
    - [First Glance](#firstglance)
- [Data](#data)
    - [2007 Data](#2007data)
    - [2017 Data](#2017data)
    - [Target Variable](#targetvariable)
    - [Reference Tables](#referencetables)
- [Extracting State Data](#extractingstatedata)
    - [New Mexico](#newmexico)
    - [Nebraska](#nebraska)
- [Initial Snapshots of the data](#initialsnapshot)
- [Null Values](#nullvalues)
    - [Full Missing Values](#fullnull)
    - [Partial Missing: High Nullity](#partialnull)
    - [Imputing null values with mean](#simpleimputer)
- [Save Clean Files](#exportclean)

# Imports   <a id=imports></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer

import autoreload
%load_ext autoreload
%autoreload 2
import python_code.Scripts as sc
import python_code.Reference as ref

### _First glance..._ <a id=firstglance></a>
These datasets contains information about each household that received SNAP benefits in 2007 and 2017.  Each household is representated by a record and goes on to record information about each member of the household.  

For our purposes, we will break this down into two states of interest: New Mexico and Nebraska.  This is due to a previous geographical analysis of emerging hot and cold spots targeting San Juan County, New Mexico as a new hot spot and Cherry County, Nebraska as a new cold spot.

# Data  <a id=data><a>

### _2007 Data_  <a id=2007data><a>
First, we are starting with the 2007 dataset downloaded from the [USDA site](https://www.fns.usda.gov/resource/snap-quality-control-data)

In [ ]:
#pip install pyreadstat #need not run this since already installed before

In [ ]:
#pip install pickle5 #need not run this since already installed before

In [ ]:
snap07 = pd.read_spss('./data/qc_pub_fy2007.sav')

In [ ]:
snap07.head(5)

In [ ]:
snap07.shape

In [ ]:
snap07.info()

In [ ]:
snap07.isnull().sum()

### _2017 Data_  <a id=2017data><a>

In [ ]:
snap17 = pd.read_csv('./data/qc_pub_fy2017.csv')

In [ ]:
snap17.head(5)

In [ ]:
snap17.info()

In [ ]:
snap17.isnull().sum()

### _Target Variable_  <a id=targetvariable><a>

In [ ]:
# 2007: 1 = Eligible, 2 = Not eligible
snap07['CAT_ELIG'].value_counts()

In [ ]:
# 2017: 0 = Not eligible, 1 = Reported eligible, 2= Recorded eligible
snap17['CAT_ELIG'].value_counts()

__Observations:__ I am going to change the target variable in both datasets to a dictionary of:
- 0 = Not Eligible
- 1 = Eligible

In [ ]:
snap07['CAT_ELIG'] = snap07['CAT_ELIG'].replace(2,0)
snap17['CAT_ELIG'] = snap17['CAT_ELIG'].replace(2,1)

In [ ]:
snap07['CAT_ELIG'].value_counts()

In [ ]:
snap17['CAT_ELIG'].value_counts()

In [ ]:
print(f'2017 dataset: {snap17.shape} VS 2007 dataset: {snap07.shape}')

__Observations:__ Less people nationally applied for SNAP benefits in 2017 as opposed to 2007.  This probably had to do with stronger national economic factors such as employment opportunities.  Also, 45 columns of features were added to the dataset.

### _Reference tables_  <a id=referencetables><a>

In [ ]:
print(f'Unit Demo, 2007 is {ref.unit07_demo.shape[0]} and 2017 is {ref.unit17_demo.shape[0]}')
print(f'Unit Assets, 2007 is {ref.unit07_assets.shape[0]} and 2017 is {ref.unit17_assets.shape[0]}')
print(f'Unit ExDed, 2007 is {ref.unit07_exded.shape[0]} and 2017 is {ref.unit17_exded.shape[0]}')
print(f'Unit Inc, 2007 is {ref.unit07_inc.shape[0]} and 2017 is {ref.unit17_inc.shape[0]}')
print(f'Pers Char, 2007 is {ref.per07_char.shape[0]} and 2017 is {ref.per17_char.shape[0]}')
print(f'Pers Inc, 2007 is {ref.per07_inc.shape[0]} and 2017 is {ref.per17_inc.shape[0]}')

__Observations:__ From 2007 to 2017, every category obtained more columns except for Expenses and Deductables.

# Extracting state data  <a id=extractingstatedata><a>

### _New Mexico_  <a id=newmexico><a>

In [ ]:
#save New Mexico records
nm07 = snap07.loc[snap07['STATE'] == 35].astype('float64')
nm07_target = nm07['CAT_ELIG']
nm07.to_csv('./data/nm07.csv',index=None)

In [ ]:
nm17 = snap17.loc[snap17['STATE'] == 35]
nm17_target = nm17['CAT_ELIG']
nm17 = nm17.drop(columns = ['STATENAME'])
nm17 = nm17.astype('float64')
nm17.to_csv('./data/nm17.csv',index=None)

In [ ]:
#New Mexico dataframe
df_nm07 = pd.read_csv('./data/nm07.csv')
df_nm07

In [ ]:
df_nm17 = pd.read_csv('./data/nm17.csv')
df_nm17

In [ ]:
df_nm07['CAT_ELIG'].value_counts()

In [ ]:
df_nm17['CAT_ELIG'].value_counts()

### _Nebraska_  <a id=nebraska><a>

In [ ]:
# Save Nebrasks records
ne07 = snap07.loc[snap07['STATE'] == 31].astype('float64')
ne07_target = ne07['CAT_ELIG']
ne07.to_csv('./data/ne07.csv',index=None)

In [ ]:
ne17 = snap17.loc[snap17['STATE'] == 31]
ne17_target = ne17['CAT_ELIG']
ne17 = ne17.drop(columns = ['STATENAME'])
ne17 = ne17.astype('float64')
ne17.to_csv('./data/ne17.csv',index=None)

In [ ]:
#Nebraska dataframe
df_ne07 = pd.read_csv('./data/ne07.csv')
df_ne07

In [ ]:
df_ne17 = pd.read_csv('./data/ne17.csv')
df_ne17

In [ ]:
df_ne07['CAT_ELIG'].value_counts()

In [ ]:
df_ne17['CAT_ELIG'].value_counts()

__Observation:__ We can see there are 1255 people who applied for SNAP in New Mexico, opposed to 791 applications in Nebraska in 2007.  

There were significantly more people in both states (Nebraska: 894, New Mexico:  964) on SNAP in 2017.  Due to a dramatic change, I suspect that it was due to more being eligible because of a change in qualifications.  

# Initial Snapshots of the data  <a id=initialsnapshot><a>

In [ ]:
sc.plot_simple_features('FSNELDER','elderly',"Number of Elderly(over age 60)")

In [ ]:
#single-female head of household - FSNGMOM
sc.plot_simple_features('FSNGMOM','single_mom',"Number of Single Mom as Head of Household")

In [ ]:
#number of children - FSNKID
sc.plot_simple_features('FSNKID','no_children',"Number of Children over 5")

In [ ]:
#number of children under 5 = FSNK5T17
sc.plot_simple_features('FSNK5T17','no_toddler',"Number of Children under 5")

In [ ]:
#number of noncitizens in unit = FSNONCIT
sc.plot_simple_features('FSNONCIT','non_citiz',"Number of Non-Citizens")

# Null Values   <a id=nullvalues><a>

This will be a long section dedicated to an initial reduction of columns by different treatments of null values.

### _Full Missing values_   <a id=fullnull><a>
There are many columns that have no data in them at all, we will remove those.

___New Mexico___

In [ ]:
with pd.option_context('display.max_rows', None):
    print(df_nm07.isnull().sum().sort_values(ascending=False)[:96])

In [ ]:
#Remove those columns.
first_cut = df_nm07.isnull().sum().sort_values(ascending=False)[:96]
first_cut_df = pd.DataFrame([first_cut])
first_cut_df.T
fc_list = list(first_cut_df.columns)

#remove them
nm07 = df_nm07.drop(fc_list,axis=1)
nm07

In [ ]:
with pd.option_context('display.max_rows', None):
    print(df_nm17.isnull().sum().sort_values(ascending=False)[:165])

In [ ]:
#Remove those columns.
first_cut = df_nm17.isnull().sum().sort_values(ascending=False)[:165]
first_cut_df = pd.DataFrame([first_cut])
first_cut_df.T
fc_list = list(first_cut_df.columns)

#remove them
nm17 = df_nm17.drop(fc_list,axis=1)
nm17

___Nebraska___

In [ ]:
with pd.option_context('display.max_rows', None):
    print(df_ne07.isnull().sum().sort_values(ascending=False)[:144])

In [ ]:
#Remove those columns.
first_cut = df_ne07.isnull().sum().sort_values(ascending=False)[:144]
first_cut_df = pd.DataFrame([first_cut])
first_cut_df.T
fc_list = list(first_cut_df.columns)

#remove them
ne07 = df_ne07.drop(fc_list,axis=1)
ne07

In [ ]:
with pd.option_context('display.max_rows', None):
    print(df_ne17.isnull().sum().sort_values(ascending=False)[:91])

In [ ]:
#Remove those columns.
first_cut = df_ne17.isnull().sum().sort_values(ascending=False)[:91]
first_cut_df = pd.DataFrame([first_cut])
first_cut_df.T
fc_list = list(first_cut_df.columns)

#remove them
ne17 = df_ne17.drop(fc_list,axis=1)
ne17

### _Partial Missing: High Nullity_   <a id=partialnull><a>

According to a paper called [The proportion of missing data should not be used to guide decisions on multiple imputation](https://www.sciencedirect.com/science/article/pii/S0895435618308710), I will be more concerned about the value of the data used to impute rather than the amount of missing information.  In order to do that, I will use a 50% cutoff of missing rows in a column to ensure the imputation method is more accurate. 

In [ ]:
dict_df = {'nm07':nm07,'nm17':nm17,'ne07':ne07,'ne17':ne17}

In [ ]:
for key, value in dict_df.items():
    print(f'50% mark for high nullitary columns:')
    print(f'{key}: {round(value.shape[0]/2)}')

In [ ]:
all_df_att = pd.DataFrame(dict_df.keys(), columns = ['name'])
all_df_att['rows'] = [value.shape[0] for key,value in dict_df.items()]
all_df_att['threshold'] = [round(value.shape[0]/2) for key, value in dict_df.items()]
all_df_att['start_col'] = [value.shape[1] for key,value in dict_df.items()]
all_df_att

In [ ]:
#New Mexico 2007
null_counts = nm07.isnull().sum()
nulls = null_counts[null_counts>628]
sc_list = list(nulls.index)

#remove them
nm07 = nm07.drop(sc_list,axis=1)
all_df_att['end_col'] = nm07.shape[1]
all_df_att

In [ ]:
#New Mexico 2017
null_counts = nm17.isnull().sum()
nulls = null_counts[null_counts>482]
sc_list = list(nulls.index)

#remove them
nm17 = nm17.drop(sc_list,axis=1)
all_df_att.loc[all_df_att['name']=='nm17',['end_col']] = nm17.shape[1]
all_df_att

In [ ]:
#Nebraska 2007
null_counts = ne07.isnull().sum()
nulls = null_counts[null_counts>396]
sc_list = list(nulls.index)

#remove them
ne07 = ne07.drop(sc_list,axis=1)
all_df_att.loc[all_df_att['name']=='ne07',['end_col']] = ne07.shape[1]
all_df_att

In [ ]:
#Nebraska 2017
null_counts = ne17.isnull().sum()
nulls = null_counts[null_counts>447]
sc_list = list(nulls.index)

#remove them
ne17 = ne17.drop(sc_list,axis=1)
all_df_att.loc[all_df_att['name']=='ne17',['end_col']] = ne17.shape[1]
all_df_att

### _Imputing null values with mean_   <a id=simpleimputer><a>
Now I will use scikitlearn imputer to fill in values for the rest of the columns with null values by accessing a script from the Py_Scripts file.  

In [ ]:
nm07 = sc.impute_df(nm07)
nm17 = sc.impute_df(nm17)
ne07 = sc.impute_df(ne07)
ne17 = sc.impute_df(ne17)

__Observation:__ Now that I have imputed the mean, we still have columns that have all zero's as their values. We will drop those columns.

In [ ]:
nm07 = sc.only_zero(nm07)
nm17 = sc.only_zero(nm17)
ne07 = sc.only_zero(ne07)
ne17 = sc.only_zero(ne17)

In [ ]:
all_df_att['orig'] = 0
all_df_att['final_col']=0

all_df_att.loc[all_df_att['name']=='nm07',['orig']] = df_nm07.shape[1]
all_df_att.loc[all_df_att['name']=='nm17',['orig']] = df_nm17.shape[1]
all_df_att.loc[all_df_att['name']=='ne07',['orig']] = df_ne07.shape[1]
all_df_att.loc[all_df_att['name']=='ne17',['orig']] = df_ne17.shape[1]

all_df_att.loc[all_df_att['name']=='nm07',['final_col']] = nm07.shape[1]
all_df_att.loc[all_df_att['name']=='nm17',['final_col']] = nm17.shape[1]
all_df_att.loc[all_df_att['name']=='ne07',['final_col']] = ne07.shape[1]
all_df_att.loc[all_df_att['name']=='ne17',['final_col']] = ne17.shape[1]

In [ ]:
all_df_att.set_index('name')

In [ ]:
quarter = (all_df_att['orig'].mean())*.25

fig, ax = plt.subplots(figsize=(20,10))
all_df_att[['orig','final_col']].plot.bar(ax=ax)
plt.xlabel('Dataset',fontsize=15)
plt.ylabel('Column count',fontsize=15)
plt.xticks([0, 1, 2,3],labels=['nm07','nm17','ne07','ne17'],rotation=360)
plt.axhline(y=quarter,linewidth=1,color='r')
plt.legend(fontsize=15)
plt.title('We are left with a quarter of the original columns \n (red line shows the quarter mark of the original column mean count)',fontsize=20)
plt.savefig('./images/final_null.png');

#  Save Clean Files   <a id=exportclean><a>

In [ ]:
nm07_target.reset_index(drop=True,inplace=True)
nm07 = nm07.assign(CAT_ELIG=nm07_target)
nm07 = nm07.astype('float64')
nm07.to_csv('./data/clean_nm07.csv',index=None)

In [ ]:
nm17_target.reset_index(drop=True,inplace=True)
nm17 = nm17.assign(CAT_ELIG=nm17_target)
nm17 = nm17.astype('float64')
nm17.to_csv('./data/clean_nm17.csv',index=None)

In [ ]:
ne07_target.reset_index(drop=True,inplace=True)
ne07 = ne07.assign(CAT_ELIG=ne07_target)
ne07 = ne07.astype('float64')
ne07.to_csv('./data/clean_ne07.csv',index=None)

In [ ]:
ne17_target.reset_index(drop=True,inplace=True)
ne17 = ne17.assign(CAT_ELIG=ne17_target)
ne17 = ne17.astype('float64')
ne17.to_csv('./data/clean_ne17.csv',index=None)